In [89]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [111]:
# Verificar se a GPU está disponível
print("GPU is available" if tf.config.list_physical_devices('GPU') else "GPU is not available")

GPU is not available


In [112]:
# Diretório onde estão armazenadas as imagens sem ser separadas por pastas
base_dir = r'C:\Users\bruna\OneDrive\Microbialdataset\images_new_dataset'

In [113]:
# Configurações
img_height, img_width = 112, 112

In [114]:
#Carregar as imagens e rótulos
images = []
labels = []

# Mapeia as classes para índices
class_names = os.listdir(base_dir)
class_indices = {name: idx for idx, name in enumerate(class_names)}

for class_name in class_names:
    class_dir = os.path.join(base_dir, class_name)
    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)
        img = load_img(img_path, target_size=(img_height, img_width))
        img_array = img_to_array(img) / 255.0  # Normalizar
        images.append(img_array)
        labels.append(class_indices[class_name])

images = np.array(images)
labels = np.array(labels)

In [115]:
# Dividir os dados em conjuntos de treinamento e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [116]:
# Converter os rótulos para one-hot encoding
y_train = to_categorical(y_train, num_classes=len(class_names))
y_test = to_categorical(y_test, num_classes=len(class_names))

In [117]:
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)


In [118]:
# Ajustar os geradores com os dados de treinamento
train_generator = train_datagen.flow(X_train, y_train, batch_size=16)

In [119]:
# Para o conjunto de teste, apenas a normalização
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(X_test, y_test, batch_size=16)

In [120]:
#Carregar o modelo base ResNet50 com pesos pré-treinados da ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

In [121]:
base_model.trainable = True  # Descongelar o modelo
for layer in base_model.layers[:143]:  # Manter as primeiras 143 camadas congeladas
    layer.trainable = False

In [122]:
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)  # Camada extra
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(class_names), activation='softmax')(x)

In [123]:
# Criar o modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [124]:
# Recompilar o modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [126]:
# Treinar o modelo
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20,
    callbacks=[early_stopping]
)

Epoch 1/20
1/1 [==============================] - 1s 1s/step - loss: 1.7298 - accuracy: 0.3750 - val_loss: 0.9353 - val_accuracy: 0.3333
Epoch 2/20
1/1 [==============================] - 1s 1s/step - loss: 0.8998 - accuracy: 0.6250 - val_loss: 0.9078 - val_accuracy: 0.3333
Epoch 3/20
1/1 [==============================] - 1s 988ms/step - loss: 1.0040 - accuracy: 0.5000 - val_loss: 0.9239 - val_accuracy: 0.3333
Epoch 4/20
1/1 [==============================] - 1s 939ms/step - loss: 0.9100 - accuracy: 0.5000 - val_loss: 0.9521 - val_accuracy: 0.3333
Epoch 5/20
1/1 [==============================] - 1s 1s/step - loss: 0.7053 - accuracy: 0.6250 - val_loss: 0.9764 - val_accuracy: 0.3333
Epoch 6/20
1/1 [==============================] - 1s 1s/step - loss: 0.6584 - accuracy: 0.7500 - val_loss: 1.0046 - val_accuracy: 0.3333
Epoch 7/20
1/1 [==============================] - 1s 1s/step - loss: 2.1615 - accuracy: 0.5000 - val_loss: 0.9786 - val_accuracy: 0.3333
